In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#house price prediction in boston

Pytorch의 MLP를 이용한 집값예측(regression)구현  
  
주요 모듈 목록  
**torch.utils.data.Dataset:** 데이터셋을 getitem method에서 한개씩 텐서형태로 반환하도록 구성하는 class  
**torch.utils.data.DataLoader:** batch processing을 위해 dataset에서 반환되는 데이터를 n개의 batch_size만큼 묶어서 반환하도록 하는 class  
**torch.nn.Module:** 딥러닝 모델을 구현한 class, forward method를 통해 입력된 텐서 데이터를 딥러닝 연산하여 결과 반환  
**torch.nn.(loss):** loss 연산을 위한 class이며 backward method를 통해 gradient계산  
**torch.optim.(optimizer):** 계산된 gradient를 update해주어 feadient descent를 진행하는 class  

Dataset class구현  
  
dataset(Housingdata.csv) : https://www.kaggle.com/datasets/altavish/boston-housing-dataset

traget: MEDV(집값)  
input feature: MEDV를 제외한 모든 값  
전체 데이터 약 500개중 400개까지 train data로 사용, 나머지는 test data  

각 data를 torch.utils.data.Dataset을 통해 하나씩 load할 수 있는 class구현  
이후 torch.utils.data.DataLoader를 통해 batch개씩 변환

torch.utils.data.Dataset의 주요 method 
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(데이터셋, 데이터경로)등을 생성  
__ len __: 해당 클래스에서 다루는 dataset의 길이를 반환하는 함수  
__ getitem __(index): index에 해당하는 데이터 하나를 tensor형태로 반환하는 함수

In [4]:
from sklearn.datasets import load_boston
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.preprocessing import MinMaxScaler


data_path = "/content/drive/MyDrive/랩세미나(new)/tutorial/open_tutorial_DL_NLP/2일차/dataset/HousingData.csv"
batch_size = 3

#torch.utils.data.Dataset을 상속하여 Dataset class선언
class myDataset(Dataset):
  #오브젝트를 선언할때 불러오는 함수, superclass(부모클래스)의 init을 실행해 주어야함
  def __init__(self, df_data) -> None:
    super().__init__()
    self.df_data = df_data #self를 사용하면 class내부에서 __init__ method만이 아닌 다른 method에서도 사용가능
    self.data_y = df_data.loc[:,["MEDV"]]
    self.data_x = df_data.drop(["MEDV"], axis=1)
      
  #list 형태의 class를 만들때 필수로 사용되는 함수, 전체 길이를 알아야 인덱싱이 가능
  def __len__(self):
    return len(self.data_y)

  #index에 해당하는 데이터를 반환해주는 함수
  def __getitem__(self, index):
    data = torch.Tensor(self.data_x.loc[index,:])
    target = torch.Tensor(self.data_y.loc[index,:])

    return data, target

#data load후 train(400개)/test(나머지)데이터를 분할
data_df = pd.read_csv(data_path).dropna()
train_data_df = data_df.loc[:400,:].reset_index()
test_data_df = data_df.loc[400:,:].reset_index()

#각 dataset을 선언
trainDataset = myDataset(train_data_df)
testDataset = myDataset(test_data_df)

#선언된 dataset을 dataloader를 통해 batch processing
trainDataloader = DataLoader(trainDataset, batch_size = batch_size)
testDataloader = DataLoader(testDataset, batch_size = batch_size)

#잘 작동하는지 test
for i in trainDataset:
  print("dataset test")
  print(i)
  break

#잘 작동하는지 test
for i in trainDataloader:
  print("data loader test")
  data = i[0]
  target = i[1]
  print(data)
  print(data.shape)
  print(target)
  print(target.shape)
  break


dataset test
(tensor([0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
        3.9690e+02, 4.9800e+00]), tensor([24.]))
data loader test
tensor([[0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
         3.9690e+02, 4.9800e+00],
        [1.0000e+00, 2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9690e+02, 9.1400e+00],
        [2.0000e+00, 2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9283e+02, 4.0300e+00]])
torch.Size([3, 14])
tensor([[24.0000],
        [21.6000],
        [34.7000]])
torch.Size([3, 1])


Deep learning 모델 구현  
torch.nn.Module을 이용하여 모델 구현  
1st hidden layer의 feature는 100개  
2nd hidden layer의 feature는 10개  
인 모델을 구현한다.  

torch.nn.Linear: perceptron의 weighted sum과 같이 linaer regression연산을 하는 calss  
torch.nn.ReLU: ReLU activation function을 수행하는 class  

torch.nn.Module의 주요 method  
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(사용할 deep learning layer, activation function, 등)등을 생성  
__ forward __(data): 입력받은 data를 딥러닝 모델을 통해 결과를 예측하여 반환하는 class  

**각 딥러닝 레이어 연산 중 차원수를 확인하고 잘 맞춰줄 것**  
참고 document(해당 사이트의 shape를 확인하고 tensor형태 결정)  
nn.Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html  
nn.ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [5]:
from torch import nn

#딥러닝 모델을 작성하기위한 모듈
class myModel(nn.Module):
  #오브젝트를 선언할 때 불러와지는 함수 일반적으로 이번 모델에서 사용될 각 레이어들이 포함됨
  def __init__(self) -> None:
      super().__init__()

      #input_feature:14,  1st_hidden: 100, 2nd_hidden: 10, output: 1의 형태에 맞는 linear layer들을 선언, activation으로 Relu사용
      self.linear1 = nn.Linear(14,100, bias=True)
      self.linear2 = nn.Linear(100,10, bias=True)
      self.linear3 = nn.Linear(10,1, bias=True)
      self.relu = nn.ReLU()
    
  #데이터를 입력받고 딥러닝 연산후 결과를 반환하는 함수
  def forward(self, x):
      #print(x.shape)
      x = self.linear1(x)
      x = self.relu(x)
      #print(x.shape)
      x = self.linear2(x)
      x = self.relu(x)
      #print(x.shape)
      x = self.linear3(x)
      x = self.relu(x)
      #print(x.shape)

      return x

#작성한 모델 선언
model = myModel()

#잘 작동하는지 test
for i in trainDataloader:
  print("model test")
  data = i[0]
  target = i[1]
  
  print("input data")
  print(data)
  print("output predict\n", model(data))
  print("ground thruth")
  print(target)
  break


model test
input data
tensor([[0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
         3.9690e+02, 4.9800e+00],
        [1.0000e+00, 2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9690e+02, 9.1400e+00],
        [2.0000e+00, 2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9283e+02, 4.0300e+00]])
output predict
 tensor([[2.1675],
        [1.9252],
        [1.3669]], grad_fn=<ReluBackward0>)
ground thruth
tensor([[24.0000],
        [21.6000],
        [34.7000]])


작성한 dataset과 model을 이용하여 딥러닝 프로세스 구현  

pytorch 딥러닝 프로세스
1. dataset, model선언
2. dataset과 model을 통한 결과 예측
3. 예측된 결과를 통해 **loss**연산 및 **loss.backward**
4. **optimizer.step()**를 사용하여 graident update

주요 오브젝트  
torch.nn.MSELoss: 예측값과 정답을 통해 MSE값을 반환하는 class  
torch.optim.Adam: Adam optimizer를 통해 gradient update를 수행하는 class

각 오브젝트의 입력과 선언은 다음 doc 참조:  
MSELoss: https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html  
Adam: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

In [6]:
from torch.optim import Adam
from torch.nn import MSELoss

#학습을 위한 optimizer와 loss function 설정
optimizer = Adam(model.parameters(), lr=0.001)
lf = MSELoss()

#100번의 에폭을 실행
for e in range(100):
  print("\n\nepoch ", e)
  epoch_loss = 0
  
  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in trainDataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()
    data = i[0]
    target = i[1]

    #결과 도출
    output = model(data)

    #loss연산
    loss = lf(output, target)
    #print(loss)

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()
  
  print("train loss", epoch_loss/len(trainDataloader))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in testDataloader:
      data = i[0]
      target = i[1]

      output = model(data)

      loss = lf(output, target)
      test_loss += loss.item()

  print("test loss", test_loss/len(testDataloader))
    




epoch  0
train loss 103.50126444271632
test loss 21.90126775418009


epoch  1
train loss 83.68431367164567
test loss 90.38706636428833


epoch  2
train loss 70.20951399121965
test loss 113.92538489614215


epoch  3
train loss 74.71994888158072
test loss 71.44948433126721


epoch  4
train loss 82.32168143966368
test loss 45.24218084130968


epoch  5
train loss 79.4304623399462
test loss 58.67536563532693


epoch  6
train loss 71.39557461440563
test loss 65.09143807206836


epoch  7
train loss 74.63659432161423
test loss 61.42538489614214


epoch  8
train loss 70.91169275215694
test loss 62.11209160940988


epoch  9
train loss 67.88339177597136
test loss 50.10132004533495


epoch  10
train loss 65.79456045003164
test loss 49.773278129952296


epoch  11
train loss 60.46799822137469
test loss 47.097924379365786


epoch  12
train loss 56.42920647774424
test loss 43.068077084209236


epoch  13
train loss 49.99406764450527
test loss 40.841173014470506


epoch  14
train loss 52.4584540741784